Importa as principais Bibliotecas

In [ ]:
import sientia_tracker.simple_tracker as simple_tracker
from sklearn.base import BaseEstimator, TransformerMixin

Configura o setup para registrar no SIENTIA

In [ ]:
# Initialize tracking
tracking_uri = "http://localhost:51107/"
username = "aignosi"
password = "aignosi"
project_name = "example_project_regression"

tracker = simple_tracker.SimpleTracker(tracking_uri,username,password)
tracker.set_project(project_name)

Setup Transformação de dados

In [ ]:
params = { "lags_features": ["temperature", "pressure", "humidity"],
            "lags": 3,
            "target": "temperature",
}


# Load dataset
data = fetch_california_housing()


class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, lags_features, params=None):
        self.params = params 
        self.data_treat = None
        self.target_variable = params["target"]
        self.variable_columns = params["lags_features"]


    def get_treated_data(self):
        return self.data_treat
    
    def get_train_data(self):
        return self.data_train

    def get_test_data(self):
        return self.data_test
    

    def fit(self, X, y=None):
        """ Fit the model according to the given training data.
        Parameters
        ----------
            X : array-like of shape (n_samples, n_features)
                The training input samples.
            y : array-like of shape (n_samples,) or (n_samples, n_outputs), default=None
                The target values (class labels in classification, real numbers in
            
        Returns
        -------
            self : object
                Returns self.
                
        Example
        -------
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size, random_state=42, shuffle=shuffle)

               """
        # your code here

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size, random_state=42, shuffle=shuffle)

        data_train = pd.concat([X_train, y_train], axis=1)
        data_test = pd.concat([X_test, y_test], axis=1)

        self.data_treat = X_train
        self.data_train = data_train
        self.data_test = data_test

        return self
    


    def transform(self, X):
        # your code here
        X = X.dropna() # example
        return X
    
    def predict(self, x):
        return self.transform(x)



preprocessor = DataPreprocessor(**params)
preprocessor.fit(data)

data_view = process_data.get_treated_data()
data_train = process_data.get_train_data()
data_test = process_data.get_test_data()


X_train = data_train[params['variable_columns']]
y_train = data_train[params['target_variable']]
X_test = data_test[params['variable_columns']]
y_test = data_test[params['target_variable']]

Setup Modelo

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate metrics
r2 = r2_score(y_test, y_pred)

Save models

In [ ]:
dataset_name= "California Housing"
inputs= params['variable_columns']
train_size = 0.8
shuffle = False


run = tracker.save_experiment(dataset_name=dataset_name, inputs=inputs)
run_id = run.info.run_id

tracker.log_params({"max_iter": 1000})
tracker.log_model(preprocessor, 'data_model')
tracker.log_model(model, "prediction_model")